<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [12]:
path = Path('medical-concept-normalization/data_collection')

In [13]:
# training data set

train_csv = path/'TwADR-L.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [14]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,24,heart beat for being too strong
2,62,feeling proper psycho
3,94,anxiety
4,133,night terrors


In [15]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-2.test.csv",header=None)
test.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,24,racing heart
2,56,persistent painful erection
3,57,hypertrophic scarring
4,94,anxiety


In [16]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [17]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [18]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [19]:
data_clas.add_test(test,label=0)


In [20]:
data_lm.save()
data_clas.save()

In [21]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'drug',
 'by',
 'on',
 'patients',
 'as',
 'that',
 'is',
 'at',
 'or',
 'mg',
 'group',
 'increased',
 'from',
 'mood',
 'study',
 'this',
 'treatment',
 'after',
 'p',
 'release',
 'an',
 'be',
 'disorder',
 ',',
 'me',
 'effect',
 'i',
 'abnormal',
 'withdrawal',
 'not',
 'decreased',
 'than',
 'compared',
 'weight',
 'anxiety',
 'groups',
 'no',
 'using',
 'are',
 'my',
 'symptoms',
 'effects',
 'inflammatory',
 'both',
 'two',
 'drugs',
 'significantly',
 'significant',
 'acute',
 'potassium',
 'skin',
 'anti',
 'between',
 'day',
 'behavior',
 'h',
 'dose',
 'all',
 'disorders',
 'blood',
 'used',
 'syndrome',
 '(',
 ')',
 'time',
 'voltaren',
 'these',
 'placebo',
 'results',
 'which',
 'had',
 '-',
 'efficacy',
 'concentration',
 'sleeplessness',
 'oral',
 'have',
 'it',
 'ds',


In [22]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [23]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.836865,4.228076,0.345810,58:13


In [25]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.015906,3.831921,0.381757,59:14


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.739721,3.787857,0.385723,59:21


In [27]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.678107,3.753765,0.389354,58:54


In [28]:
learn.save_encoder('TwADR_pubmed_fold2_first')

In [29]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold2_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.199482,6.566236,0.078571,01:19


In [30]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.261480,5.557024,0.092857,01:24


In [31]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.702356,5.199523,0.121429,03:04


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.409135,4.722571,0.192857,03:17
1,5.167701,4.568894,0.214286,03:20


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.931015,4.382425,0.214286,03:15
1,4.750274,4.298354,0.221429,03:16


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.558293,4.124784,0.250000,03:19
1,4.369778,4.063530,0.257143,03:07


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.154916,3.991494,0.271429,03:23
1,4.004797,3.942941,0.285714,03:18


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.858638,3.807827,0.292857,03:06
1,3.704192,3.701417,0.314286,03:07


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.512677,3.649511,0.335714,03:18
1,3.439916,3.623492,0.335714,03:26
2,3.254142,3.550891,0.364286,03:13
3,3.091109,3.529598,0.357143,03:17


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.897871,3.584560,0.328571,03:07
1,2.848092,3.525436,0.335714,03:19
2,2.709912,3.568906,0.350000,03:03
3,2.541939,3.543825,0.342857,03:19


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.376066,3.525526,0.350000,03:17
1,2.360226,3.571804,0.350000,03:14
2,2.186786,3.591262,0.371429,03:12
3,2.038167,3.550888,0.378571,03:08


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.884116,3.617758,0.371429,03:19
1,1.858101,3.573722,0.400000,03:24
2,1.715435,3.585540,0.378571,03:06
3,1.586367,3.593527,0.400000,03:16


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.462405,3.681417,0.364286,03:23
1,1.454741,3.731477,0.378571,03:10
2,1.355643,3.740706,0.371429,03:22
3,1.223897,3.794671,0.371429,03:14


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.142988,3.693445,0.400000,03:20
1,1.159347,3.792310,0.392857,03:12
2,1.078509,3.822785,0.378571,03:12
3,0.976213,3.820003,0.364286,03:22


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.880659,3.872225,0.378571,03:18
1,0.895826,3.951263,0.371429,03:22
2,0.847691,3.968568,0.357143,03:06
3,0.791557,3.947050,0.364286,03:15


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.767189,4.125974,0.342857,03:08
1,0.737578,3.958614,0.371429,03:20


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.698178,4.056379,0.414286,03:14
1,0.668639,4.035613,0.414286,03:08


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.652032,4.072524,0.364286,03:24


In [47]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold2')

In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.630835,4.248214,0.385714,03:19


In [49]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.571112,4.245546,0.357143,03:23


In [50]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.580646,4.311902,0.371429,03:20


In [51]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.533584,4.340794,0.350000,03:27


# Part three: Predict on the test dataset

In [52]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [53]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[2191, 24, 56, 551, 978, 2191, 1545, 1545, 189, 1545, 217, 1521, 2147, 233, 2147, 1136, 301, 13, 13, 13, 945, 418, 445, 1597, 582, 908, 609, 720, 744, 744, 749, 749, 749, 887, 755, 1718, 792, 792, 2125, 978, 824, 426, 265, 1184, 1846, 974, 974, 974, 978, 1599, 1019, 359, 359, 1718, 1925, 1099, 2032, 1526, 1099, 1136, 561, 1184, 1184, 1157, 1373, 1265, 1265, 1295, 1295, 852, 1295, 1308, 1373, 1340, 1340, 1507, 1973, 2191, 2191, 1822, 2110, 582, 978, 712, 1701, 1526, 1529, 1062, 1545, 426, 109, 1597, 1597, 1815, 1767, 1597, 1610, 1136, 1073, 1526, 1925, 1718, 1531, 1718, 1718, 1718, 1718, 1718, 1718, 1545, 1768, 712, 13, 1787, 1787, 13, 393, 1196, 418, 1810, 2191, 1073, 2191, 233, 1196, 233, 233, 233, 1545, 1942, 1822, 923, 1767, 1230, 1948, 824, 1099, 1295, 2147, 2032, 2194, 2191, 393]
[13, 24, 56, 57, 94, 113, 189, 189, 189, 189, 217, 224, 233, 233, 233, 265, 301, 393, 393, 393, 396, 418, 445, 452, 582, 605, 609, 720, 744, 745, 749, 749, 749, 749, 755, 791, 792, 792, 813, 819, 

In [54]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

56
0.3916083916083916
